# Chapter 9: Portfolio Theory

Understanding portfolio theory is very important in learning finance. It is well
known that don't put all your eggs in one basket, that is, it is a great idea to diversify
away your risk. However, very few know the implied assumption behind such a
famous idiom. In this chapter, we will discuss various risk measures for individual
stocks or portfolios, such as Sharpe ratio, Treynor ratio, Sortino ratio, how to
minimize portfolio risk based on those measures (ratios), how to set up an objective
function, how to choose an efficient portfolio for a given set of stocks, and how to
construct an efficient frontier. Our focus is on how to apply portfolio theory by using
real-world data. For instance, today we have $2 million cash and plan to purchase
IBM and Walmart stocks. If we have 30% invested in the first one and the rest in the
second, what is our portfolio risk? What is the least risky portfolio that we could
form based on those two stocks? How about 10 or 500 stocks? In this chapter, the
following topics will be covered:

• Introduction to portfolio theory

• A 2-stock portfolio

• N-stock portfolio

• Correlation versus diversification effect

• Producing a return matrix

• Generating an optimal portfolio based on Sharpe ratio, Treynor ratio, and
Sortinor ratio

• Constructing an efficient frontier

• Modigliani and Modigliani performance measure (M2 measure)